<img src="dh_logo.png" align="right" width="50%">

## PCA

#### Caso tivessemos $5$ dimensões, cinco filmes nesse caso, para plotar, sería necessário fazer a redução de dimensões para $3$ ou para $2$. Calculando as [componentes principais](https://towardsdatascience.com/the-best-tool-for-better-recommendations-systems-e57142b45f11) para o caso de $5$ filmes, faríamos da seguinte maneira:

In [238]:
movie_rating_dropna = movie_rating.dropna()
data_pca = movie_rating_dropna.drop(['Usuario'], 
                                    axis = 1
                                   ).values

In [239]:
data_pca

array([[0., 5., 1., 5., 3., 5.],
       [1., 5., 1., 5., 5., 5.],
       [2., 4., 3., 2., 1., 2.],
       [5., 5., 1., 5., 5., 5.],
       [6., 1., 3., 2., 5., 4.]])

In [240]:
pca = PCA(n_components = 3)
principalComponents  = pca.fit_transform(data_pca)

In [241]:
principalComponents

array([[-3.38323955,  0.32584291, -0.57286375],
       [-2.405704  , -1.31725096, -0.73285748],
       [ 0.36993687,  4.16200378,  0.7162813 ],
       [ 0.49426563, -2.71035765,  1.550396  ],
       [ 4.92474104, -0.46023807, -0.96095607]])

## Escalonamento dos dados

#### Ao utilizar a **Distância Euclidiana**, é preciso tomar cuidado com a escala das variáveis. 

#### Quando vamos mensurar a distância temos que ter cuidado ao comparar medidas de similaridade, como por exemplo Metros versus Idade, ou Metros versus Centímetros, ou Dias versus Anos.

Exemplo: Uma criança de $2$ a $4$ anos tem em geral $74$ centímetros de altura, porém, a comparativamente entre $2$ e $74$ há uma ordem de grandeza de diferença de escala, pois $74$ é um número muito maior do que $2$, isso poderia enviesar o nosso modelo fazendo-o interpretar erroneamente a distância entre essas duas medidas. Porém, já que uma criança de $2$ a $4$ anos pode ter $74$ centímetros de altura, precisamos fazer com que essas medidas sejam interpretadas pelo modelo como estando na mesma escala, assim não estariamos enviesando o modelo. Uma forma muito utilizada de fazer isso é como a seguir:

<img src="min-max-normalisation.jpeg" align="center" width="40%">  

#### Entendendo a escala acima:

In [242]:
df = pd.DataFrame([[1, 54], 
                   [3, 88], 
                   [5, 110], 
                   [7, 150]
                  ], 
                  columns = ["idade", "centimetros"]
                 )
df

,idade,centimetros
0,1,54
1,3,88
2,5,110
3,7,150


#### Nossa fórmula ficaria como visto a seguir:

```python
nova_idade = (idade(indice) - min(idade)) / (max(idade) - min(idade))
```

#### Vamos adotar biblioteca de `scaling` do `Sklearn`:

In [243]:
from sklearn.preprocessing import MinMaxScaler

df = MinMaxScaler().fit_transform(df)
df

array([[0.        , 0.        ],
       [0.33333333, 0.35416667],
       [0.66666667, 0.58333333],
       [1.        , 1.        ]])

## Medidas de dissimilaridade

#### Nem sempre utilizaremos o critério de distância para mensurar a dissimilaridade entre uma observação e outra. Em alguns momentos a distância eucliciana pode não ser a melhor escolha. Por exemplo, se você gostaria de fazer recomendações de itens comprados em uma loja `online`, pode-se dizer que seria melhor indicar os produtos que as pessoas compram em comum.

#### Observe a matriz a seguir:

In [249]:
compras = pd.DataFrame([[1, 1, 2], 
                        [4, 1, 5], 
                        [15, 0, 2], 
                        [8, 0, 8], 
                        [1, 0, 0]
                       ], 
                       columns = ['Meia', 'Sapatos', 'Sandalias'], 
                       index = ['Maria', 'Joana', 'Roberto', 'Joao', 'Manuel']
                      )
print(compras)
print('')
compras = compras.values

print(compras)

pd.DataFrame(np.linalg.norm(compras - compras[ : , None], 
                            axis = -1
                           )
            )

         Meia  Sapatos  Sandalias
Maria       1        1          2
Joana       4        1          5
Roberto    15        0          2
Joao        8        0          8
Manuel      1        0          0

[[ 1  1  2]
 [ 4  1  5]
 [15  0  2]
 [ 8  0  8]
 [ 1  0  0]]


,0,1,2,3,4
0,0.000000,4.242641,14.035669,9.273618,2.236068
1,4.242641,0.000000,11.445523,5.099020,5.916080
2,14.035669,11.445523,0.000000,9.219544,14.142136
3,9.273618,5.099020,9.219544,0.000000,10.630146
4,2.236068,5.916080,14.142136,10.630146,0.000000


#### Podemos notar na matriz de distância acima que `Maria` $(0)$ está mais próxima a `Manuel` $(4)$, porém, os mesmos não compraram os mesmos itens, e pior, o `Manuel` não é um comprador nem de sapatos e nem de sandálias. Podemos considerar que se fossemos trabalhar com distância euclidiana nesse caso, estaríamos assumindo que procuramos pessoas que não são compradoras `online` do `site`, ou seja, não são clientes ativos. 

#### A cliente `Joana` $(1)$ está longe da `Maria` $(0)$, mas as mesmas compraram os mesmos itens na loja, isso não parece muito sensato dado que queremos recomendar produtos que uma compra para a outra.

#### Porém, gostaríamos de validar pessoas que compraram produtos similares e assim identificar gostos em comum dessas pessoas. Sendo assim poderíamos trabalhar com a [medida da Correlação](https://towardsdatascience.com/collaborative-filtering-based-recommendation-systems-exemplified-ecbffe1c20b1), que idenfica o quanto os itens que as pessoas compraram andam juntos, na mesma direção até $+ 1$, ou opostas umas as outras até $-1$.

#### Por mais que a nossa correlação seja $+$ ou $-$, as compras do usuária `Maria` $(0)$ andam com as compras da usuária `Joana` $(1)$, então as mesmas andam comprando o mesmo item mesmo que em quantidades diferentes. Sendo assim podemos acreditar que quando indicarmos um produto que a `Maria` comprou para a `Joana`, há uma grande possibilidade da mesma se interessar pelo item, maximizando nossas vendas no `site`.

#### Transformamos as compras de novo em um `Dataframe`.

In [250]:
compras = pd.DataFrame(compras)
print(compras)

    0  1  2
0   1  1  2
1   4  1  5
2  15  0  2
3   8  0  8
4   1  0  0


#### A medida de correlação pede que os compradores sejam colunas ao invés de linhas, é a forma como o pandas funciona para correlacionar os compradores.

In [251]:
compras = compras.T
print(compras)

   0  1   2  3  4
0  1  4  15  8  1
1  1  1   0  0  0
2  2  5   2  8  0


In [248]:
compras.corr()

,0,1,2,3,4
0,1.000000,0.693375,-0.389885,0.500000,-0.500000
1,0.693375,1.000000,0.393217,0.970725,0.277350
2,-0.389885,0.393217,1.000000,0.602549,0.992434
3,0.500000,0.970725,0.602549,1.000000,0.500000
4,-0.500000,0.277350,0.992434,0.500000,1.000000


#### Acima podemos identificar uma relação entre a `Joana` $(1)$ e a `Maria` $(0)$ que compram produtos parecidos mesmo que de forma correlacionada negativamente. O interessante é que a `Maria` $(0)$ e `Joana` $(1)$ estão mais negativamente correlacionadas do que a `Maria` $(0)$ e o `Manuel` $(4)$. Isso significa que a importância da `Maria` $(0)$ com `Joana` $(1)$ é alta, e isso nos diz que podemos recomendar itens que uma compra para a outra.

## Filtragem Baseada em Itens

#### Nós podemos também trabalhar com [Filtragem baseada em Itens](https://www.analyticsvidhya.com/blog/2015/08/beginners-guide-learn-content-based-recommender-systems/). Esse tipo de recomendação é utilizada com bases grandes. Imagine uma base muito grande, ela pode trazer um pouco de dificuldade ao fazer recomendações para usuários, uma vez que os mesmos podem variar demais em relação aos produtos comprados, uma vez que os itens podem não ter mudado com tanta freqüência assim. 

#### Nesses casos, para bases muito grandes, a filtragem de itens é recomendada. Alguns pontos que valem a pena comentarmos em relação à construção de sistemas de Filtragem:

- Cálculos antecipados

<img src="Download-Time.jpeg" align="center" width="10%">  

- Pré-computar itens

<img src="3-2-database-free-download-png.png" align="center" width="10%">  

- Média ponderada com itens similares

<img src="media_ponderada.png" align="center" width="10%">  

- As comparações não mudarão com frequencia

<img src="comparacoes_frequencia.jpeg" align="center" width="10%">  

- O algoritmo roda de tempos em tempos (menor custo computacional)

<img src="batch-logo.png" align="center" width="10%">  

#### A implementação desse metodo vai além do escopo desse material, porém, vamos comentar algumas propriedades desse algoritmo.

#### Como o nome diz, iremos fazer uma recomendação baseada em similaridade de itens e por esse motivo devemos ter em mente que como os itens mudam com pouca freqüência esse algoritmo não precisa rodar sempre.

#### O que é muito diferente do caso de recomendação por usuários, pois, o tempo todo novos usuários estão assistindo a novos filmes, mas, nem sempre temos filmes novos no catálogo para os usuários assistirem. Por esse motivo não precisamos fazer rodar nosso algoritmo com a base toda o tempo todo.

#### Podemos salvar em uma base de dados as similaridade entre todos os filmes, dadas as notas de usuários, com seus respectivos pesos para encontramos em nosso banco de dados.

#### Como nem sempre aparecem filmes novos no catálogo posso rodar esse algoritmo semanalmente para recomendar novos filmes aos meus usuários. 

#### Algumas diferenças entre Filtragem por Usuários e por Itens. 

#### Usuários:

- **1**: Implementação mais simples;

- **2**: É indicada para bancos menores, conjuntos densos;

- **3**: Por vezes, apenas o cálculo de dissimiladidade já faz um bom trabalho.

#### Itens:

- **1**: Mais rápido, pois os dados já estão em memória; 

- **2**: Recomendado para bases maiores, com conjuntos esparsos, porém aplicável a conjuntos densos.

#### Falando sobre o conjunto de dados:
    
- Conjuntos densos são conjuntos em que muitos usuários deram nota para os filmes;

- Conjuntos esparsos são conjuntos em que os usuários deram nota para itens diferentes.

In [252]:
movie_rating_item = movie_rating.drop(['Usuario'], 
                                      axis = 1
                                     ).transpose().reset_index()

In [253]:
movie_rating_item.columns = ['Filmes'] + movie_rating['Usuario'].tolist()

In [254]:
movie_rating_item

,Filmes,Izabela,Thais,Lorena,Leandro,Uliane,Isis,Marcos
0,Unnamed: 0,0.0,1.0,2.0,3.0,4.0,5.0,6.0
1,Orgulho & Preconceito,5.0,5.0,4.0,NaN,2.0,5.0,1.0
2,Velozes e Furiosos,1.0,1.0,3.0,5.0,NaN,1.0,3.0
3,O Senhor dos anéis,5.0,5.0,2.0,NaN,3.0,5.0,2.0
4,Star Wars,3.0,5.0,1.0,2.0,NaN,5.0,5.0
5,De volta para o futuro,5.0,5.0,2.0,4.0,1.0,5.0,4.0


In [255]:
dataset_filmes = movie_rating_item.drop(['Filmes'], 
                                        axis = 1
                                       ).dropna(axis = 1).values

In [256]:
similaridades_filmes = pd.DataFrame(np.linalg.norm(dataset_filmes - dataset_filmes[ : , None], 
                                                   axis = -1
                                                  )
                                   )
similaridades_filmes

,0,1,2,3,4,5
0,0.000000,8.366600,5.196152,7.549834,5.196152,6.708204
1,8.366600,0.000000,7.280110,2.236068,5.385165,3.605551
2,5.196152,7.280110,0.000000,7.071068,6.633250,7.071068
3,7.549834,2.236068,7.071068,0.000000,3.741657,2.000000
4,5.196152,5.385165,6.633250,3.741657,0.000000,2.449490
5,6.708204,3.605551,7.071068,2.000000,2.449490,0.000000


In [257]:
movie_rating_item.Filmes

0                Unnamed: 0
1     Orgulho & Preconceito
2        Velozes e Furiosos
3        O Senhor dos anéis
4                 Star Wars
5    De volta para o futuro
Name: Filmes, dtype: object

In [259]:
filmes_vistos = movie_rating.loc[movie_rating['Usuario'] == user
                                ].dropna(axis = 1).drop(['Usuario'],
                                                        axis = 1
                                                       ).keys().tolist()

In [260]:
filmes_vistos

['Unnamed: 0', 'Velozes e Furiosos', 'Star Wars', 'De volta para o futuro']

In [261]:
filmes_nao_vistos = list(set(movie_rating) - set(filmes_vistos))

In [262]:
filmes_nao_vistos

['Orgulho & Preconceito', 'Usuario', 'O Senhor dos anéis']

In [263]:
movie_rating_item.loc[(movie_rating_item.Filmes.isin(filmes_vistos) )][['Filmes', 'Leandro']]

,Filmes,Leandro
0,Unnamed: 0,3.0
2,Velozes e Furiosos,5.0
4,Star Wars,2.0
5,De volta para o futuro,4.0


In [264]:
similaridades_filmes = 1 / (1 + similaridades_filmes)

In [265]:
similaridades_filmes

,0,1,2,3,4,5
0,1.000000,0.106762,0.161390,0.116961,0.161390,0.129732
1,0.106762,1.000000,0.120771,0.309017,0.156613,0.217129
2,0.161390,0.120771,1.000000,0.123899,0.131006,0.123899
3,0.116961,0.309017,0.123899,1.000000,0.210897,0.333333
4,0.161390,0.156613,0.131006,0.210897,1.000000,0.289898
5,0.129732,0.217129,0.123899,0.333333,0.289898,1.000000


In [266]:
similaridades_filmes[0]

0    1.000000
1    0.106762
2    0.161390
3    0.116961
4    0.161390
5    0.129732
Name: 0, dtype: float64

In [268]:
movie_rating_item_leandro = pd.concat([movie_rating_item.loc[(movie_rating_item.Filmes.isin(filmes_vistos))][['Filmes', 'Leandro']], 
                                       similaridades_filmes[0], similaridades_filmes[2]
                                      ],
                                      axis = 1
                                     ).dropna()

In [269]:
movie_rating_item_leandro = movie_rating_item_leandro.rename(columns = {'Leandro':'Notas dadas por Leandro', 
                                                                        0:'Sim. Org e Prec', 
                                                                        2: 'Sim. Senhor d. A.'
                                                                       }
                                                            )

In [270]:
movie_rating_item_leandro

,Filmes,Notas dadas por Leandro,Sim. Org e Prec,Sim. Senhor d. A.
0,Unnamed: 0,3.0,1.000000,0.161390
2,Velozes e Furiosos,5.0,0.161390,1.000000
4,Star Wars,2.0,0.161390,0.131006
5,De volta para o futuro,4.0,0.129732,0.123899


In [274]:
movie_rating_item_leandro['Sim. Org e Prec x Notas'] = movie_rating_item_leandro['Notas dadas por Leandro'
                                                                                ] * movie_rating_item_leandro['Sim. Org e Prec'] 

In [275]:
movie_rating_item_leandro

,Filmes,Notas dadas por Leandro,Sim. Org e Prec,Sim. Senhor d. A.,Sim. Org e Prec x Notas,Sim. Senhor d. A. x Notas
0,Unnamed: 0,3.0,1.000000,0.161390,3.000000,0.484171
2,Velozes e Furiosos,5.0,0.161390,1.000000,0.806952,5.000000
4,Star Wars,2.0,0.161390,0.131006,0.322781,0.262012
5,De volta para o futuro,4.0,0.129732,0.123899,0.518928,0.495597


In [276]:
movie_rating_item_leandro['Sim. Senhor d. A. x Notas'] = movie_rating_item_leandro['Notas dadas por Leandro'
                                                                                  ] * movie_rating_item_leandro['Sim. Senhor d. A.'] 

In [277]:
movie_rating_item_leandro

,Filmes,Notas dadas por Leandro,Sim. Org e Prec,Sim. Senhor d. A.,Sim. Org e Prec x Notas,Sim. Senhor d. A. x Notas
0,Unnamed: 0,3.0,1.000000,0.161390,3.000000,0.484171
2,Velozes e Furiosos,5.0,0.161390,1.000000,0.806952,5.000000
4,Star Wars,2.0,0.161390,0.131006,0.322781,0.262012
5,De volta para o futuro,4.0,0.129732,0.123899,0.518928,0.495597


In [279]:
movie_rating_item_leandro['Sim. Org e Prec x Notas'].sum() / movie_rating_item_leandro['Sim. Org e Prec'].sum()

3.200426717545168

In [280]:
movie_rating_item_leandro['Sim. Senhor d. A. x Notas'].sum() / movie_rating_item_leandro['Sim. Senhor d. A.'].sum()

4.407116920775932